In [1]:
import sys
import os

# 프로젝트 루트 디렉토리 경로를 추가
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
import jax
import jax.numpy as jnp
#disable jax JIT
jax.config.update("jax_disable_jit", True)

from JAxtar.hash import dataclass_hashing, dataclass_hashing_batch
from puzzle.slidepuzzle import SlidePuzzle
from heuristic.slidepuzzle_heuristic import SlidePuzzleHeuristic
from JAxtar.hash import HashTable

In [3]:
puzzle = SlidePuzzle(4)
sample = jax.vmap(puzzle.get_initial_state)(key=jax.random.split(jax.random.PRNGKey(0),2))
table = HashTable.make_lookup_table(puzzle.State, 1, int(1e8))

idx, table_idx, found = jax.vmap(HashTable.check, in_axes=(None,0))(table, sample)
print(idx, table_idx, found)

ConcretizationTypeError: Abstract tracer value encountered where concrete value is expected: traced array with shape int32[].
pad_width argument of jnp.pad
The error occurred while tracing the function <lambda> at /home/tinker/JAxtar/JAxtar/hash.py:133 for cond. This value became a tracer due to JAX operations on these lines:

  operation a:i32[] = convert_element_type[new_dtype=int32 weak_type=True] b
    from line /home/tinker/JAxtar/JAxtar/hash.py:18:10 (to_uint32)

  operation a:i32[] = convert_element_type[new_dtype=int32 weak_type=True] b
    from line /home/tinker/JAxtar/JAxtar/hash.py:18:10 (to_uint32)

  operation a:i32[] = convert_element_type[new_dtype=int32 weak_type=True] b
    from line /home/tinker/JAxtar/JAxtar/hash.py:20:8 (to_uint32)

  operation a:i32[] = convert_element_type[new_dtype=int32 weak_type=True] b
    from line /home/tinker/JAxtar/JAxtar/hash.py:20:8 (to_uint32)

  operation a:i32[] = convert_element_type[new_dtype=int32 weak_type=True] b
    from line /h